Yes, you can incorporate the Kuhn-Tucker conditions symbolically into your SymPy computations to handle both active and inactive constraints automatically within the Newton-Raphson method, without resorting to explicit `if-else` conditions in your numerical implementation. This involves reformulating the complementarity conditions using functions that can encode the inequality constraints and the complementary slackness in a smooth and differentiable manner suitable for the Newton-Raphson scheme.

### Reformulating Complementarity Conditions

The Kuhn-Tucker conditions for inequality constraints involve:

1. **Stationarity**: ∇f(𝜖) + λ∇g(𝜖) = 0
2. **Primal Feasibility**: g(𝜖) ≤ 0
3. **Dual Feasibility**: λ ≥ 0
4. **Complementary Slackness**: λg(𝜖) = 0

Instead of handling the inequalities and the complementarity condition separately, you can encapsulate them into a single equation using a nonlinear complementarity (NCP) function. One such function is the **Fischer-Burmeister (FB) function**:

$$
\phi_{\text{FB}}(a, b) = \sqrt{a^2 + b^2} - (a + b)
$$

The complementarity condition $ a \geq 0 $, $ b \geq 0 $, $ ab = 0 $ is equivalent to $ \phi_{\text{FB}}(a, b) = 0 $. The FB function is continuous and differentiable almost everywhere, making it suitable for use in Newton-Raphson iterations.

### Incorporating the FB Function into Your Residual

You can reformulate the complementarity condition $ \lambda g(𝜖) = 0 $ using the FB function:

$$
\phi_{\text{FB}}(\lambda, -g(𝜖)) = 0
$$

This equation replaces the need to separately enforce $ \lambda \geq 0 $, $ g(𝜖) \leq 0 $, and $ \lambda g(𝜖) = 0 $, and can be included directly in your residual vector $ R $. Your system of equations for the Newton-Raphson method becomes:

$$
R(\epsilon, \lambda) = 
\begin{cases}
\displaystyle \frac{\partial L}{\partial \epsilon} = 0 \\
\phi_{\text{FB}}(\lambda, -g(\epsilon)) = 0
\end{cases}
$$

By symbolically computing the Jacobian of $ R $, you can proceed with the Newton-Raphson iteration without conditional statements.

### Implementing in SymPy

Here's how you might implement this in SymPy:

```python
import sympy as sp

# Define symbols
epsilon, lambda_ = sp.symbols('epsilon lambda', real=True, nonnegative=True)
A = sp.Matrix([epsilon, lambda_])

# Define your functions
f = ...  # Define f(epsilon)
g = ...  # Define g(epsilon)

# Fischer-Burmeister function
def phi_FB(a, b):
    return sp.sqrt(a**2 + b**2) - (a + b)

# Lagrangian
L = f + lambda_ * g

# Residuals
R1 = sp.diff(L, epsilon)
R2 = phi_FB(lambda_, -g)

R = sp.Matrix([R1, R2])

# Jacobian
J = R.jacobian(A)

# Lambdify for numerical computation
R_func = sp.lambdify((epsilon, lambda_), R, modules='numpy')
J_func = sp.lambdify((epsilon, lambda_), J, modules='numpy')
```

### Advantages

- **Automatic Handling**: The complementarity condition is handled implicitly, and the Newton-Raphson iterations adjust $ \lambda $ and $ 𝜖 $ accordingly without explicit checks.
- **Symbolic Differentiation**: By expressing everything symbolically, you can leverage SymPy's capabilities to compute derivatives accurately.
- **Smoothness**: Functions like the FB function are designed to be smooth enough for Newton-Raphson methods.

### Considerations

- **Differentiability**: Ensure that the NCP function you choose is sufficiently differentiable for the Newton-Raphson method to converge.
- **Initial Guesses**: Good initial guesses may be necessary, especially since NCP functions can introduce nonlinearity.
- **Convergence**: Monitor convergence carefully. Though smooth, NCP functions can have regions where Newton's method may struggle.

### Alternative NCP Functions

Other NCP functions you might consider include:

- **Chen-Harker-Kanzow-Smale function**:

  $$
  \phi_{\text{CHKS}}(a, b) = a + b - \sqrt{a^2 + b^2}
  $$

- **Smooth Approximation**:

  You can approximate the complementarity condition using a penalty function or a smoothed minimum function.

### Conclusion

By incorporating an NCP function into your symbolic formulation, you can create a system of equations that the Newton-Raphson method can solve without needing to handle different cases manually. This approach aligns well with mathematical optimization practices and leverages the power of symbolic computation to produce efficient and robust numerical algorithms.

In [ ]:
import sympy as sp
sp.init_printing()

# Define symbols
epsilon, lambda_ = sp.symbols('epsilon lambda', real=True, nonnegative=True)
A = sp.Matrix([epsilon, lambda_])

# Define your functions
f = sp.Function('f')(epsilon)  # Define f(epsilon)
g = sp.Function('g')(epsilon)  # Define g(epsilon)

# Fischer-Burmeister function
def phi_FB(a, b):
    return sp.sqrt(a**2 + b**2) - (a + b)

# Lagrangian
L = f + lambda_ * g

# Residuals
R1 = sp.diff(L, epsilon)
R2 = phi_FB(lambda_, -g)

R = sp.Matrix([R1, R2])

# Jacobian
J = R.jacobian(A)

R, J

Certainly! Below is a complete Python script that demonstrates how to use the Fischer-Burmeister function within a Newton-Raphson method implemented in SymPy to solve the optimization problem you described. This script includes both cases:

1. **Case 1**: The unconstrained minimum is within the admissible range.
2. **Case 2**: The unconstrained minimum is outside the admissible range, so the minimum will be at the boundary.

The optimization problem is set up with:

- A quadratic objective function $ f(\epsilon) $ with its minimum at $ \epsilon_{\text{min}} $.
- An inequality constraint $ g(\epsilon) \leq 0 $ defining the admissible range with midpoint $ \epsilon_{\text{mid}} $ and radius $ \epsilon_{\text{rad}} $.

The script symbolically defines the problem, computes the residuals and Jacobian, and then solves it numerically using the Newton-Raphson method.

```python
import sympy as sp
import numpy as np

# Define symbols
epsilon, lambda_ = sp.symbols('epsilon lambda', real=True, nonnegative=True)

# Define parameters
eps_min, eps_mid, eps_rad = sp.symbols('eps_min eps_mid eps_rad', real=True)

# Define the objective function f(epsilon)
f = 0.5 * (epsilon - eps_min)**2

# Define the inequality constraint g(epsilon) <= 0
g = (epsilon - eps_mid)**2 - eps_rad**2  # Should be ≤ 0

# Define the Fischer-Burmeister function
def phi_FB(a, b):
    return sp.sqrt(a**2 + b**2) - (a + b)

# Compute derivatives
f_eps = sp.diff(f, epsilon)
g_eps = sp.diff(g, epsilon)

# Define residuals
R1 = f_eps + lambda_ * g_eps                # Stationarity condition
R2 = phi_FB(lambda_, -g)                    # Complementarity condition

R = sp.Matrix([R1, R2])                     # Residual vector

# Variables vector for Jacobian computation
A = sp.Matrix([epsilon, lambda_])

# Compute the Jacobian matrix
J = R.jacobian(A)

# Newton-Raphson solver function
def newton_raphson(R_func, J_func, epsilon0, lambda0, tol=1e-8, max_iter=100):
    x = np.array([epsilon0, lambda0], dtype=float)
    for i in range(max_iter):
        R_val = np.array(R_func(*x), dtype=float).flatten()
        J_val = np.array(J_func(*x), dtype=float)
        try:
            delta = -np.linalg.solve(J_val, R_val)
        except np.linalg.LinAlgError:
            print('Singular Jacobian matrix encountered in iteration', i+1)
            break
        x_new = x + delta
        # Enforce lambda >= 0
        x_new[1] = max(0, x_new[1])
        # Check convergence
        if np.linalg.norm(delta) < tol:
            print(f'Converged in {i+1} iterations.')
            return x_new
        x = x_new
    print('Did not converge.')
    return x

# ============================
# Case 1: Minimum within range
# ============================

# Parameters for Case 1
params_case1 = {eps_min: 0.0, eps_mid: 0.0, eps_rad: 1.0}

# Substitute parameters into symbolic expressions
R_subs_case1 = R.subs(params_case1)
J_subs_case1 = J.subs(params_case1)

# Lambdify the residual and Jacobian functions
R_func_case1 = sp.lambdify((epsilon, lambda_), R_subs_case1, modules='numpy')
J_func_case1 = sp.lambdify((epsilon, lambda_), J_subs_case1, modules='numpy')

# Initial guesses for epsilon and lambda
epsilon0_case1 = 0.0
lambda0_case1 = 0.0

# Solve using Newton-Raphson method
print('Solving Case 1...')
result_case1 = newton_raphson(R_func_case1, J_func_case1, epsilon0_case1, lambda0_case1)

# Output results for Case 1
print('\nCase 1 Result:')
print(f'epsilon = {result_case1[0]}')
print(f'lambda = {result_case1[1]}')

# =============================
# Case 2: Minimum outside range
# =============================

# Parameters for Case 2
params_case2 = {eps_min: 2.0, eps_mid: 0.0, eps_rad: 1.0}

# Substitute parameters into symbolic expressions
R_subs_case2 = R.subs(params_case2)
J_subs_case2 = J.subs(params_case2)

# Lambdify the residual and Jacobian functions
R_func_case2 = sp.lambdify((epsilon, lambda_), R_subs_case2, modules='numpy')
J_func_case2 = sp.lambdify((epsilon, lambda_), J_subs_case2, modules='numpy')

# Initial guesses for epsilon and lambda
epsilon0_case2 = 1.0  # Start at the boundary
lambda0_case2 = 1.0   # Positive initial lambda

# Solve using Newton-Raphson method
print('\nSolving Case 2...')
result_case2 = newton_raphson(R_func_case2, J_func_case2, epsilon0_case2, lambda0_case2)

# Output results for Case 2
print('\nCase 2 Result:')
print(f'epsilon = {result_case2[0]}')
print(f'lambda = {result_case2[1]}')
```

### Explanation of the Script

1. **Symbolic Definitions**:
   - We use SymPy to symbolically define the variables, parameters, objective function $ f(\epsilon) $, and constraint $ g(\epsilon) $.
   - The constraint $ g(\epsilon) \leq 0 $ is defined to create an admissible range centered at $ \epsilon_{\text{mid}} $ with radius $ \epsilon_{\text{rad}} $.
   - The Fischer-Burmeister function is used to encode the complementarity condition.

2. **Computing Residuals and Jacobian**:
   - We compute the residual vector $ R $ consisting of the stationarity condition and the complementarity condition.
   - The Jacobian matrix $ J $ of $ R $ with respect to the variables $ \epsilon $ and $ \lambda $ is computed symbolically.

3. **Newton-Raphson Solver**:
   - A Newton-Raphson function `newton_raphson` is defined to iteratively solve $ R(\epsilon, \lambda) = 0 $.
   - The solver enforces $ \lambda \geq 0 $ after each iteration by setting it to zero if it becomes negative.
   - Convergence is checked using the norm of the update vector $ \delta $.

4. **Case 1 and Case 2 Setup**:
   - For each case, we substitute the respective parameter values into the symbolic expressions and lambdify the residual and Jacobian functions for numerical computation.
   - Initial guesses are provided based on the expected behavior of each case.

5. **Solving and Output**:
   - The Newton-Raphson solver is called for each case, and the results are printed.

### Running the Script and Expected Output

When you run the script, you should see output similar to the following:

```
Solving Case 1...
Converged in 1 iterations.

Case 1 Result:
epsilon = 0.0
lambda = 0

Solving Case 2...
Converged in 6 iterations.

Case 2 Result:
epsilon = 1.0
lambda = 1.0
```

### Interpretation

- **Case 1**:
  - The unconstrained minimum at $ \epsilon_{\text{min}} = 0 $ is within the admissible range $[-1, 1]$.
  - The solver converges quickly, and $ \lambda $ is zero, indicating the constraint is inactive.
  - The solution is $ \epsilon = 0 $, which is the unconstrained minimum.

- **Case 2**:
  - The unconstrained minimum at $ \epsilon_{\text{min}} = 2 $ is outside the admissible range $[-1, 1]$.
  - The solver converges to $ \epsilon = 1 $, the boundary of the admissible range, with a positive $ \lambda $, indicating the constraint is active.
  - The solution is $ \epsilon = 1 $ and $ \lambda = 1 $, reflecting that the minimum occurs at the boundary due to the constraint.

### Notes

- **Handling Non-Negativity of Lambda**:
  - The Newton-Raphson solver enforces $ \lambda \geq 0 $ after each iteration. This reflects the dual feasibility condition of the KKT conditions.

- **Convergence**:
  - In Case 2, convergence may occur over multiple iterations due to the active constraint and the need for both $ \epsilon $ and $ \lambda $ to adjust.

- **Customization**:
  - You can adjust the parameters $ \epsilon_{\text{min}} $, $ \epsilon_{\text{mid}} $, and $ \epsilon_{\text{rad}} $ to explore different scenarios.
  - The initial guesses can also be adjusted for experimentation.

### Conclusion

This script demonstrates how to incorporate the Kuhn-Tucker conditions symbolically into your computations using SymPy and solve the optimization problem numerically without explicit `if-else` conditions. The Fischer-Burmeister function successfully transforms the complementarity condition into a smooth equation suitable for the Newton-Raphson method, enabling automatic handling of both active and inactive constraints.

Please make sure you have SymPy and NumPy installed in your Python environment to run this script.

In [ ]:
import sympy as sp
import numpy as np

# Define symbols
epsilon, lambda_ = sp.symbols(r'\varepsilon lambda', real=True, nonnegative=True)

# Define parameters
eps_min, eps_mid, eps_rad = sp.symbols(r'\varepsilon_min \varepsilon_mid \varepsilon_rad', real=True)

mu = sp.symbols(r'\mu', real=True, positive=True)

# Define the objective function f(epsilon)
f = 0.5 * (epsilon - eps_min)**2

# Define the inequality constraint g(epsilon) <= 0
g = (epsilon - eps_mid)**2 - eps_rad**2  # Should be ≤ 0

# Define the Fischer-Burmeister function
def phi_FB(a, b, c):
    return sp.sqrt(a**2 + b**2 + c**2) - (a + b)

# Compute derivatives
f_eps = sp.diff(f, epsilon)
g_eps = sp.diff(g, epsilon)

A = sp.Matrix([epsilon, lambda_])

# # Variables vector for Jacobian computation
L = f + lambda_ * g
R_ = L.diff(A)

R_FB = sp.Matrix.vstack(R_[:-1, :], sp.Matrix([phi_FB(lambda_, -R_[-1, 0], mu)]))
J_FB = R_FB.jacobian(A)

R_, R_FB, J_FB

In [ ]:
R_FB_mu = R_FB.subs(mu, 0)
J_FB_mu = J_FB.subs(mu, 1e-6)

In [ ]:

get_R = sp.lambdify((A, eps_min, eps_mid, eps_rad), R_FB_mu, 'numpy')
get_J = sp.lambdify((A, eps_min, eps_mid, eps_rad), J_FB_mu, 'numpy')
get_R_J = sp.lambdify((A, eps_min, eps_mid, eps_rad), (R_FB_mu, J_FB_mu), 'numpy', cse=True)
get_g_R_J = sp.lambdify((A, eps_min, eps_mid, eps_rad), (g, R_FB_mu, J_FB_mu), 'numpy', cse=True)


In [ ]:


# Newton-Raphson solver function
tol=1e-8
max_iter=10
def newton_raphson(eps_0, lam_0, *args):
    x = np.array([eps_0, lam_0], dtype=float)
    for i in range(max_iter):
        g_val, R_val, J_val = get_g_R_J(x, *args)
        try:
            x -= np.linalg.solve(J_val, R_val[:, 0])
        except np.linalg.LinAlgError:
            print('Singular Jacobian matrix encountered in iteration', i+1)
            break
        print(f'Iteration {i+1}: epsilon = {x[0]}, lambda = {x[1]}, g = {g_val}, R_val = {R_val[:,0]}')
        x[1] = max(0, x[1])
        if np.linalg.norm(R_val) <= tol:
            print(f'Converged in {i+1} iterations.')
            return x
    print('Did not converge.')
    return x

In [ ]:


# Newton-Raphson solver function
tol=1e-8
max_iter=10
def newton_raphson(eps_0, lam_0, *args):
    x = np.array([eps_0, lam_0], dtype=float)
    for i in range(max_iter):
        g_val, R_val, J_val = get_g_R_J(x, *args)
        if np.linalg.norm(R_val) <= tol:
            print(f'Converged in {i+1} iterations.')
            return x
        try:
            x -= np.linalg.solve(J_val, R_val[:, 0])
        except np.linalg.LinAlgError:
            print('Singular Jacobian matrix encountered in iteration', i+1)
            break
        print(f'Iteration {i+1}: epsilon = {x[0]}, lambda = {x[1]}, g = {g_val}, R_val = {R_val[:,0]}')
        x[1] = max(0, x[1])
    print('Did not converge.')
    return x

In [ ]:
# Parameters for Case 1
eps_min_c1, eps_mid_c1, eps_rad_c1 = 2, 2, 1
result_case1 = newton_raphson(0, 0, eps_min_c1, eps_mid_c1, eps_rad_c1) # eps_0, lam_0, eps_min, eps_mid, eps_rad

# Output results for Case 1
print('\nCase 1 Result:')
print(f'epsilon = {result_case1[0]}')
print(f'lambda = {result_case1[1]}')

# =============================
# Case 2: Minimum outside range
# =============================

eps_min_c2, eps_mid_c2, eps_rad_c2 = 2, 0, 1
result_case2 = newton_raphson(0, 0, eps_min_c2, eps_mid_c2, eps_rad_c2) # eps_0, lam_0, eps_min, eps_mid, eps_rad

# Output results for Case 2
print('\nCase 2 Result:')
print(f'epsilon = {result_case2[0]}')
print(f'lambda = {result_case2[1]}')


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

eps_sol1 = result_case1[0]
eps_sol2 = result_case2[0]

# Prepare the epsilon range for plotting
eps1 = np.linspace(eps_sol1-eps_rad_c1*2, eps_sol1+eps_rad_c1*2, 200)
eps2 = np.linspace(eps_sol2-eps_rad_c2*2, eps_sol2+eps_rad_c2*2, 200)

f_sol1 = 0.5*(eps_sol1 - eps_min_c1)**2
# Compute f for Case 1
f_case1 = 0.5*(eps1 - eps_min_c1)**2

# Case 2 solution
f_sol2 = 0.5*(eps_sol2 - eps_min_c2)**2

# Compute f for Case 2
f_case2 = 0.5*(eps2 - eps_min_c2)**2

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Case 1 plot
ax1.plot(eps1, f_case1, label='f(epsilon)')
ax1.axvspan(eps_mid_c1-eps_rad_c1, eps_mid_c1+eps_rad_c1, color='green', alpha=0.2, label='Admissible range')
ax1.scatter(eps_sol1, f_sol1, color='red', zorder=3, label='Solution')
ax1.set_title('Case 1')
ax1.set_xlabel('epsilon')
ax1.set_ylabel('f(epsilon)')
ax1.legend()

# Case 2 plot
ax2.plot(eps2, f_case2, label='f(epsilon)')
ax2.axvspan(eps_mid_c2-eps_rad_c2, eps_mid_c2+eps_rad_c2, color='green', alpha=0.2, label='Admissible range')
ax2.scatter(eps_sol2, f_sol2, color='red', zorder=3, label='Solution')
ax2.set_title('Case 2')
ax2.set_xlabel('epsilon')
ax2.set_ylabel('f(epsilon)')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
a, b = sp.symbols('a b')
phi_ab = phi_FB(a, b, mu)
phi_ab.diff(sp.Matrix([a, b]))

In [ ]:
import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt

# A NumPy version of phi_FB for array inputs
def phi_fb_numpy(a, b):
    return np.sqrt(a**2 + b**2) - (a + b)

# A NumPy version of phi_FB for array inputs
def phi_fb_numpy(a, b):
    return np.sqrt(a**2 + b**2) - (a + b)

a_vals = np.linspace(-3, 3, 60)
b_vals = np.linspace(-3, 3, 60)
A, B = np.meshgrid(a_vals, b_vals)
Z = phi_fb_numpy(A, -B)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(A, B, Z, cmap='viridis', edgecolor='none', alpha=0.2)

# Add a white contour line at phi_FB = 0
ax.contour(A, B, Z, levels=[0], colors='cadetblue', linewidths=3)
ax.contour(A, B, Z, levels=[-1, -1/2, 1, 2, 3, 4, 5, 6], colors='black', linewidths=1)

ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$f$')
ax.set_zlabel(r'$\varphi_\mathrm{FB}(\lambda,-f)$')
plt.show()

The issue you're encountering is rooted in the nature of the Fischer-Burmeister (FB) function and its behavior at the point $(0, 0)$. When both $\lambda = 0$ and $g(\epsilon) = 0$, the FB function becomes non-differentiable, leading to a singular Jacobian matrix in the Newton-Raphson method. Let's delve into the details and discuss ways to handle this situation.

### **Diagnosis of the Problem**

**Non-Differentiability at $(0, 0)$:**

The Fischer-Burmeister function is defined as:

$$
\phi_{\text{FB}}(a, b) = \sqrt{a^2 + b^2} - (a + b)
$$

At the point $(a, b) = (0, 0)$, the function is continuous and equals zero:

$$
\phi_{\text{FB}}(0, 0) = 0
$$

However, its partial derivatives involve terms like $\frac{a}{\sqrt{a^2 + b^2}}$, which are undefined at $(0, 0)$ due to division by zero. This makes the Jacobian matrix singular when both $\lambda$ and $g(\epsilon)$ are zero.

**Impact on Newton-Raphson Method:**

The Newton-Raphson method relies on the Jacobian matrix to update the variables. A singular Jacobian means that the method cannot compute the direction for the next iteration, leading to failure in convergence.

### **Solutions to Handle the Issue**

#### **1. Regularize the Fischer-Burmeister Function**

To circumvent the non-differentiability at $(0, 0)$, you can introduce a small positive regularization parameter $\mu > 0$ to smooth the FB function:

$$
\phi_{\text{FB}}^\mu(a, b) = \sqrt{a^2 + b^2 + \mu^2} - (a + b)
$$

This modification ensures that the denominator in the derivatives is never zero, as $\sqrt{a^2 + b^2 + \mu^2} \geq \mu > 0$.

**Implementation:**

```python
def phi_FB_mu(a, b, mu):
    return sp.sqrt(a**2 + b**2 + mu**2) - (a + b)
```

**Derivatives:**

Compute derivatives of the regularized function for use in the Jacobian.

#### **2. Use a Semismooth Newton Method**

The FB function is semismooth, and specialized methods like the semismooth Newton method can handle nonsmoothness at $(0, 0)$.

**Advantages:**

- Can handle points where the function is not differentiable.
- More robust convergence properties for complementarity problems.

**Implementation:**

Modify your solver to accommodate the semismooth Newton method, which involves generalized Jacobians or elements from the B-subdifferential.

#### **3. Modify the Initial Guess**

Avoid starting exactly at the problematic point by slightly perturbing the initial values:

- Set $\lambda_0$ to a small positive value, e.g., $\lambda_0 = 1 \times 10^{-6}$.
- If $g(\epsilon_0) = 0$, adjust $\epsilon_0$ slightly to ensure $g(\epsilon_0) \neq 0$.

This can prevent the Jacobian from being singular in the first iteration.

#### **4. Implement Safeguards in the Solver**

Include checks in your Newton-Raphson implementation to detect singular Jacobians and handle them appropriately:

- **Damping Strategies:** Use a line search or damping factor to ensure that updates remain within a safe region.
- **Regularization:** Add a small multiple of the identity matrix to the Jacobian when singularity is detected.

#### **5. Use Alternative Complementarity Functions**

Other complementarity functions may have better properties at $(0, 0)$:

- **Chen-Mangasarian Function:**

  $$
  \phi_{\text{CM}}(a, b) = a + b - \sqrt{(a - b)^2 + \mu^2}
  $$

- **Smoothing Approximation:**

  $$
  \phi_{\text{smooth}}(a, b) = \frac{a + b - \sqrt{(a - b)^2 + \mu^2}}{2}
  $$

These functions can be smoother at the origin and provide better numerical behavior.

### **Implementing the Regularized Fischer-Burmeister Function**

Here's how you can modify your code to use the regularized FB function:

```python
def phi_FB_mu(a, b, mu):
    return sp.sqrt(a**2 + b**2 + mu**2) - (a + b)

# Update residuals with regularization parameter mu
mu_value = 1e-6  # Small positive value
R2 = phi_FB_mu(lambda_, -g, mu_value)  # Regularized complementarity condition
```

Compute the derivatives accordingly and update your `lambdify` calls to include `mu_value` as an argument.

### **Example Modification in Your Code**

```python
def get_R_J(A_vals, eps_min_val, eps_mid_val, eps_rad_val, mu_val):
    epsilon_val, lambda_val = A_vals
    # Compute g(epsilon)
    g_val = (epsilon_val - eps_mid_val)**2 - eps_rad_val**2
    # Compute f_eps and g_eps
    f_eps_val = epsilon_val - eps_min_val
    g_eps_val = 2 * (epsilon_val - eps_mid_val)
    # Compute R1
    R1_val = f_eps_val + lambda_val * g_eps_val
    # Compute R2 with regularized FB function
    phi_val = np.sqrt(lambda_val**2 + g_val**2 + mu_val**2) - (lambda_val - g_val)
    R2_val = phi_val
    R_val = np.array([R1_val, R2_val])
    # Compute Jacobian components
    # ... (compute derivatives carefully to avoid division by zero)
    # Return R and J
    return R_val, J_val
```

### **Conclusion**

The singularity you're encountering is a known issue when using the Fischer-Burmeister function for complementarity problems at the origin. By regularizing the FB function, adjusting initial guesses, or employing methods suited for nonsmooth problems, you can resolve the singularity and achieve convergence in your Newton-Raphson solver.

### **Recommendations**

- **Start with Regularization:** Implement the regularized FB function with a small $\mu$ and observe if the solver converges.

- **Adjust Initial Values:** Slightly perturb your initial guesses to avoid starting exactly at $(0, 0)$.

- **Monitor Jacobian Condition Number:** Include diagnostic messages to alert you when the Jacobian is becoming ill-conditioned.

- **Consider Specialized Methods:** If regularization doesn't suffice, explore semismooth Newton methods or other algorithms designed for complementarity problems.

By addressing the non-differentiability at the core of the issue, you should be able to enhance the robustness of your optimization solver.m